In [ ]:
# example from
#https://plot.ly/python/v3/surface-triangulation/

In [3]:
#!pip install plyfile

In [34]:
#!pip install chart-studio

In [35]:
#import plotly.plotly as py
import chart_studio.plotly as py

import numpy as np
import matplotlib.cm as cm
import plotly.graph_objs as go


In [9]:
from plyfile import PlyData, PlyElement

import urllib.request
req = urllib.request.Request('http://people.sc.fsu.edu/~jburkardt/data/ply/chopper.ply')
#opener = urllib.build_opener()
#f = opener.open(req)
response=urllib.request.urlopen(req)
#data = response.read().decode('utf-8') 
plydata = PlyData.read(response)

In [11]:
for element in plydata.elements:
    print(element)

nr_points=plydata.elements[0].count
nr_faces=plydata.elements[1].count

element vertex 1066
property float x
property float y
property float z
element face 2094
property list uchar int vertex_indices


In [14]:
points=np.array([plydata['vertex'][k] for k in range(nr_points)])
points[0]

x,y,z=zip(*points)

faces=[plydata['face'][k][0] for k in range(nr_faces)]
faces[0]

array([0, 1, 2], dtype=int32)

In [23]:
def tri_indices(simplices):
    #simplices is a numpy array defining the simplices of the triangularization
    #returns the lists of indices i, j, k

    return ([triplet[c] for triplet in simplices] for c in range(3))
    
def plotly_trisurf(x, y, z, simplices, colormap=cm.RdBu, plot_edges=None):
    #x, y, z are lists of coordinates of the triangle vertices 
    #simplices are the simplices that define the triangularization;
    #simplices  is a numpy array of shape (no_triangles, 3)
    #insert here the  type check for input data

    points3D=np.vstack((x,y,z)).T
    tri_vertices=map(lambda index: points3D[index], simplices)# vertices of the surface triangles     
    zmean=[np.mean(tri[:,2]) for tri in tri_vertices ]# mean values of z-coordinates of 
                                                      #triangle vertices
    min_zmean=np.min(zmean)
    max_zmean=np.max(zmean)
    facecolor=[map_z2color(zz,  colormap, min_zmean, max_zmean) for zz in zmean]
    I,J,K=tri_indices(simplices)

    triangles=go.Mesh3d(x=x,
                     y=y,
                     z=z,
                     facecolor=facecolor,
                     i=I,
                     j=J,
                     k=K,
                     name=''
                    )

    if plot_edges is None:# the triangle sides are not plotted 
        return [triangles]
    else:
        #define the lists Xe, Ye, Ze, of x, y, resp z coordinates of edge end points for each triangle
        #None separates data corresponding to two consecutive triangles
        lists_coord=[[[T[k%3][c] for k in range(4)]+[ None]   for T in tri_vertices]  for c in range(3)]
        Xe, Ye, Ze=[reduce(lambda x,y: x+y, lists_coord[k]) for k in range(3)]

        #define the lines to be plotted
        lines=go.Scatter3d(x=Xe,
                        y=Ye,
                        z=Ze,
                        mode='lines',
                        line=dict(color= 'rgb(50,50,50)', width=1.5)
               )
        return [triangles, lines]

In [24]:
def map_z2color(zval, colormap, vmin, vmax):
    #map the normalized value zval to a corresponding color in the colormap

    if vmin>vmax:
        raise ValueError('incorrect relation between vmin and vmax')
    t=(zval-vmin)/float((vmax-vmin))#normalize val
    R, G, B, alpha=colormap(t)
    return 'rgb('+'{:d}'.format(int(R*255+0.5))+','+'{:d}'.format(int(G*255+0.5))+\
           ','+'{:d}'.format(int(B*255+0.5))+')'

In [30]:
axis = dict(
showbackground=True,
backgroundcolor="rgb(230, 230,230)",
gridcolor="rgb(255, 255, 255)",
zerolinecolor="rgb(255, 255, 255)",
    )
    
layout = go.Layout(
         title='Moebius band triangulation',
         width=800,
         height=800,
         scene=dict(
         xaxis=dict(axis),
         yaxis=dict(axis),
         zaxis=dict(axis),
        aspectratio=dict(
            x=1,
            y=1,
            z=0.5
        ),
        )
        )

In [37]:
data3=plotly_trisurf(x,y,z, faces, colormap=cm.RdBu, plot_edges=None)

title="Trisurf from a PLY file<br>"+\
                "Data Source:<a href='http://people.sc.fsu.edu/~jburkardt/data/ply/airplane.ply'> [1]</a>"

noaxis=dict(showbackground=False,
            showline=False,
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            title=''
          )

fig3 = go.Figure(data=data3, layout=layout)
fig3['layout'].update(dict(title=title,
                           width=1000,
                           height=1000,
                           scene=dict(xaxis=noaxis,
                                      yaxis=noaxis,
                                      zaxis=noaxis,
                                      aspectratio=dict(x=1, y=1, z=0.4),
                                      camera=dict(eye=dict(x=1.25, y=1.25, z= 1.25)
                                     )
                           )
                     ))

from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot
init_notebook_mode(connected=True)
#py.iplot(fig3, filename='Chopper-Ply-cls')
iplot(fig3, filename='Chopper-Ply-cls')